# Recreating Existing Gov Nets Data Processing Using Python

As my first capstone project for the Springboard Data Science Career Track course, I am preparing an R/shiny app to display the networks governance networks which I gathered for Vermont EPSCoR.

The data is on github: https://github.com/wmirecon/Water_Quality_Governance_Networks


## Data Reprocessing

These data were previously cleaned using R and R/dplyr. For the purposes of the Springboard course, I am reprocessing the data using Python and Python/pandas

## Processing Steps:

### Privately performed
The following steps will be performed locally and not stored to project's Github repository. This is done because the raw survey returns contain an extensive ammount of personally-identifiable information (PII). In accordance with the requirements of the University of Vermont's Institutional Review Board (IRB), under whose auspices these data were collected, the PII is scrubbed from the data prior to it being posted to Github. This steps will be performed here, and their scripting maintained, but all processing will be done locally.

1. Remove extraneous data about the individuals who responded to the survey, including names and contact information. Only organizational names will remain. (done prior to loading into python)
3. Construct a key to transform organizational names into anonymous identifies in accordance with the assignment process described in the codebook (done prior to loading into python)
3. Clean column headers
4. Replace organizational names for those organizations that responded to the survey with anonymous identifiers
5. Break data set into pieces: a) header questions (org size, budget, etc.) b) bimodal networks (domains, policy tools, and action arenas, accountability mechanisms), c) links for major subnetworks
6. Transpose the dataframe for the major subnet links and replace organizational names with anonymous IDs

### Publicly performed steps
Once the above steps have been completed, the data will be anonymized. The three data set portions described in step 5, above, are moved to the Github repository and final processing is performed:

1. Separate the links from all the major subnetworks into separate data frames, one for each subnetwork and re-transpose to obtain a non-square network (NSQ) network matrix
2. Save NSQ matrices and process matrices into edge list datasets that can be read into R/igraph
3. Separate bimodal data into four separate datasets, one each for domains, policy tools, action arenas, and accountability mechanisms

### Establish workspace

In [118]:
# setup packages

import pandas as pd
import numpy as np

In [160]:
# setup function to convert NSQs to edge lists

"""
Generally, network software, including igraph in both R and Python, can't read NSQs.
If you have a matrix, it must be a square matrix for unimodal data or a bimodal matrix,
or your program will fail to read it correctly. But raw survey data returns non-square
unimodal matrices with no clear way to transform them into a square matrix. But network
data can also be stored as edgelists. Indeed, at least in R/igraph, these are better for
creating igraph network objects as the command allows for including node attribute data
from edgelists but not from matrices. I haven't checked if this is also the case in
Python/igraph, but, since they're made by the same people, I'm presuming it is the case
there too. Edgelists also highly portable across a range of SNA platforms, more so even
than matrices are.
"""

def nsq_to_el( nsq ):
    import pandas as pd
    el = []                    # a list to store edges in; will be converted to a data frame
    
    # obtain row and column counts for use in for-loops
    n_init = nsq.shape[0]
    n_respond = nsq.shape[1]
    
    # loop through the cells of the dataframe that contain link data
    # links are indicated by values 1 and 2
    # if a link is found, store the initiating and responding orgs (row and column names) in a tuple
    # and append the tuple to the list of edges
    for init in range(n_init):
        for respond in range(n_respond):
            if ( nsq.iloc[init, respond] in ('1','2') ):
                edge = ( nsq.iloc[init, 0], nsq.iloc[0,respond])
                el.append(edge)
    
    # convert edgelist list into a data frame
    el_df = pd.DataFrame.from_records(el, columns = ['from','to'])
    return(el_df)

In [165]:
# read in data sets

# main data: survey returns and keys for translating names to acronyms and anonymous acronyms
prep_data = pd.read_csv(r'C:\Users\srsch\Documents\In Class Notes\Springboard\capstone projects\gov nets python data processing\data prep y2.csv',
                      header = None)


# keys are converted to dictionaries to all for faster processing
# org names and acronyms that were readily recognizable to respondents were used in the survey to limit cognitive burden
# but names are too long and are cumbersome to use, particularly during net mapping and external acronyms are not unique
# a set in internal acronyms, which are unique was devised for internal processing
# the internal acronyms were then subplanted by anonymous ids to protect respondents' identities

# convert names to internal acronyms
name_key = pd.read_csv(r'C:\Users\srsch\Documents\In Class Notes\Springboard\capstone projects\gov nets python data processing\name key.csv')
name_key = dict(zip(name_key.org_name, name_key.int_acronym))

# convert external acronyms to internal acronyms
id_key = pd.read_csv(r'C:\Users\srsch\Documents\In Class Notes\Springboard\capstone projects\gov nets python data processing\acronym key.csv')
id_key = dict(zip(id_key.ext_id, id_key.int_id))

# convert internal acronyms to anonymous ids
anon_key = pd.read_csv(r'C:\Users\srsch\Documents\In Class Notes\Springboard\capstone projects\gov nets python data processing\anonymization key.csv')
anon_key = dict(zip(anon_key.int_id, anon_key.anon_id))


# header data, to be used as first ROW of the appropriate dataset
subnet_headers_df = pd.read_csv(r'C:\Users\srsch\Documents\In Class Notes\Springboard\capstone projects\gov nets python data processing\subnet headers.csv',
                            header = None)
subnet_headers = list(subnet_headers_df[0])

bimodal_headers_df = pd.read_csv(r'C:\Users\srsch\Documents\In Class Notes\Springboard\capstone projects\gov nets python data processing\bimodal headers.csv',
                            header = None)
bimodal_headers = list(bimodal_headers_df[0])




In [143]:
# review data sets to ensure read in properly and formated as desired.

#print(prep_data.iloc[:,0:6].head(10))
#print(prep_data.loc[0])
#print(name_key)
#print(subnet_headers)

## THIS CONTENT SUPPRESSED AND OMITTED FOLLOWING COMPLETION AS IT CONTAINS PII ##

### Private Processing

In [166]:
# build list to assign as new row names
# due to duplicate columns names in the network data, these MUST be left not as column index but as row 0 of the df

header = ['full_org_name','program','ext_acronym','org_type']

for col_head in bimodal_headers:
    header.append(col_head)

for col_head in subnet_headers:
    header.append(col_head)



"""
an error occured where the data and the header information were of different lengths
the following code is where I tracked down where the two diverged
I checked corresponding values in the first row of the dataset and the list of column names until I find divergence
what I discovered was that the raw data still had an organization that had been removed from the header data
this organization is a known error in the raw data; it had been included in the survey erroneously
after confirming that this was the source of the error, I removed the offending data from the raw data and re-ran
x = 360
print(len(header))
print(len(prep_data.loc[0]))
print(prep_data.get_value(0,x, takeable = True))
print(header[x])
"""

# assign column names to first row
prep_data.loc[0] = header

#print(prep_data.iloc[:,0:9].head(10))


#### Assign Internal Acronyms and Anonymous ID's to respondents' organization names

In [167]:
# number of errors made using the name difficult
# a small change was made in official acronyms to allow that to be used as a key (only two values overlapped; one was changed)
prep_data['int_id'] = prep_data[2].map(id_key)

# report all lines where the dictionary failed
prep_data.loc[prep_data['int_id'].isnull()]
# once only the initial line reports, the data is ready for further processing

,0,1,2,3,4,5,6,7,8,9,...,1104,1105,1106,1107,1108,1109,1110,1111,1112,int_id
0,full_org_name,program,ext_acronym,org_type,Winooski_River_watershed,Missisquoi_River_watershed,Tool_Providers__Establish_conservation_easements,Tool_Providers__Enforce_regulation_compliance,Tool_Providers__Communicate_public_information,Tool_Providers__Administer_permits,...,IRDA,IRDA,IRDA,IRDA,Franklin Watershed Committee,Franklin Watershed Committee,Franklin Watershed Committee,Franklin Watershed Committee,Franklin Watershed Committee,NaN


In [168]:
# assign anonymous id's and check for failures
prep_data['anon_id'] = prep_data.int_id.map(anon_key)
prep_data.loc[prep_data.int_id.isnull()]

,0,1,2,3,4,5,6,7,8,9,...,1105,1106,1107,1108,1109,1110,1111,1112,int_id,anon_id
0,full_org_name,program,ext_acronym,org_type,Winooski_River_watershed,Missisquoi_River_watershed,Tool_Providers__Establish_conservation_easements,Tool_Providers__Enforce_regulation_compliance,Tool_Providers__Communicate_public_information,Tool_Providers__Administer_permits,...,IRDA,IRDA,IRDA,Franklin Watershed Committee,Franklin Watershed Committee,Franklin Watershed Committee,Franklin Watershed Committee,Franklin Watershed Committee,NaN,NaN


In [169]:
# replace existing ID's with anonymous ids
prep_data[0] = prep_data.anon_id
prep_data.set_value(0,0,'anon_id')
prep_data.drop([1,2,3], axis = 1, inplace = True)
prep_data.drop('int_id', axis = 1, inplace = True)
prep_data.drop('anon_id', axis = 1, inplace = True)
prep_data.head(10)

,0,4,5,6,7,8,9,10,11,12,...,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112
0,anon_id,Winooski_River_watershed,Missisquoi_River_watershed,Tool_Providers__Establish_conservation_easements,Tool_Providers__Enforce_regulation_compliance,Tool_Providers__Communicate_public_information,Tool_Providers__Administer_permits,Tool_Providers__Make_grants,Tool_Providers__Commit_to_cost_sharing,Tool_Providers__Make_loans_and_or_guarantees,...,IRDA,IRDA,IRDA,IRDA,IRDA,Franklin Watershed Committee,Franklin Watershed Committee,Franklin Watershed Committee,Franklin Watershed Committee,Franklin Watershed Committee
1,n281600,Yes,Yes,No,No,Yes,No,Yes,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,g321601,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,...,1,1,1,1,1,2,2,2,2,2
3,n454701,Yes,Yes,No,No,No,No,No,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,w231401,Yes,No,No,No,Yes,No,Yes,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,w241216,Yes,No,Yes,Yes,Yes,Yes,No,Yes,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,g321612,Yes,Yes,No,Yes,No,Yes,No,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,w241206,Yes,No,No,Yes,Yes,Yes,No,Yes,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,m241201,No,Yes,No,Yes,Yes,Yes,No,No,No,...,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN
9,w341203,Yes,No,No,Yes,Yes,Yes,Yes,Yes,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Break data into sections

In [170]:
# separate out bimodal data sections

# identify last column of bi-modal data sets; found at:
print(prep_data.iloc[:,[63,64,65]].head())

bimodal_sets = prep_data.drop(list(range(68,prep_data.shape[1] + 3)), axis = 1)
bimodal_sets.head(10)

                                                  66  \
0  Accountability_Mechanism_Professional_codes_of...   
1                                                Yes   
2                                                Yes   
3                                                Yes   
4                                                 No   

                                                  67           68  
0  Accountability_Mechanism_Feedback_from_peer_or...  AAFM -Other  
1                                                 No          NaN  
2                                                Yes            2  
3                                                Yes          NaN  
4                                                 No          NaN  


,0,4,5,6,7,8,9,10,11,12,...,58,59,60,61,62,63,64,65,66,67
0,anon_id,Winooski_River_watershed,Missisquoi_River_watershed,Tool_Providers__Establish_conservation_easements,Tool_Providers__Enforce_regulation_compliance,Tool_Providers__Communicate_public_information,Tool_Providers__Administer_permits,Tool_Providers__Make_grants,Tool_Providers__Commit_to_cost_sharing,Tool_Providers__Make_loans_and_or_guarantees,...,Domain___Forestry,Domain___Development,Accountability_Mechanism_Feedback_from_federal...,Accountability_Mechanism_Feedback_garnered_fro...,Accountability_Mechanism_Outcomes_of_court_cas...,Accountability_Mechanism_Feedback_from_shareho...,Accountability_Mechanism_Feedback_from_consume...,Accountability_Mechanism_Direct_oversight_from...,Accountability_Mechanism_Professional_codes_of...,Accountability_Mechanism_Feedback_from_peer_or...
1,n281600,Yes,Yes,No,No,Yes,No,Yes,No,No,...,No,No,No,No,No,No,No,No,Yes,No
2,g321601,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,...,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes
3,n454701,Yes,Yes,No,No,No,No,No,No,No,...,No,Yes,Yes,No,No,Yes,Yes,No,Yes,Yes
4,w231401,Yes,No,No,No,Yes,No,Yes,No,No,...,No,Yes,Yes,No,No,No,Yes,No,No,No
5,w241216,Yes,No,Yes,Yes,Yes,Yes,No,Yes,Yes,...,No,Yes,Yes,Yes,No,No,No,No,No,Yes
6,g321612,Yes,Yes,No,Yes,No,Yes,No,No,No,...,No,No,No,Yes,Yes,No,Yes,No,No,Yes
7,w241206,Yes,No,No,Yes,Yes,Yes,No,Yes,No,...,Yes,Yes,Yes,Yes,No,No,No,No,No,Yes
8,m241201,No,Yes,No,Yes,Yes,Yes,No,No,No,...,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,Yes
9,w341203,Yes,No,No,Yes,Yes,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes


In [171]:
# save bimodal set into the public repository
bimodal_sets.to_csv('data/bimodal data sets 2015 python public start.csv', index = False, header = False)

#### Acquire network subnets dataset and assign anonymous IDs to column headers to approximate a network matrix

In [126]:
# limit data to just network data sets, fill NaN's with 0's, transpose, and change to anon_ids
combined_subnets = prep_data.drop(list(range(4,68)), axis = 1)
combined_subnets.fillna(0, inplace = True)
combined_subnets_t = combined_subnets.transpose()

# the data is transposed so that a new column may be made that adds in data on subnet labels and for assigning anon_ids
# this is typically easier to do in columns than rows. transposing allows for doing these operations as columns
# final products will be transposed back to produce properly-configured NSQ's

# replace first new first column with internal ids and check for errors
combined_subnets_t['int_id'] = combined_subnets_t.iloc[:,0].map(name_key)
combined_subnets_t.loc[combined_subnets_t['int_id'].isnull()]

,0,1,2,3,4,5,6,7,8,9,...,85,86,87,88,89,90,91,92,93,int_id
0,anon_id,n281600,g321601,n454701,w231401,w241216,g321612,w241206,m241201,w341203,...,n261609,n454800,m264700,n261607,n261503,n261501,n261606,n161605,g431305,NaN


In [127]:
# replace first new first column with anonymous ids and check for errors 
combined_subnets_t['anon_id'] = combined_subnets_t.int_id.map(anon_key)
combined_subnets_t.loc[combined_subnets_t['anon_id'].isnull()]

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,int_id,anon_id
0,anon_id,n281600,g321601,n454701,w231401,w241216,g321612,w241206,m241201,w341203,...,n454800,m264700,n261607,n261503,n261501,n261606,n161605,g431305,NaN,NaN


In [128]:
# replace anon_ids as the first column (necessary for network matrix structure) and remove trailing ids columns

combined_subnets_t[0] = combined_subnets_t.anon_id
combined_subnets_t.set_value(0,0,'anon_id')
combined_subnets_t.drop('int_id', axis = 1, inplace = True)
combined_subnets_t.drop('anon_id', axis = 1, inplace = True)
combined_subnets_t.head(10)

,0,1,2,3,4,5,6,7,8,9,...,84,85,86,87,88,89,90,91,92,93
0,anon_id,n281600,g321601,n454701,w231401,w241216,g321612,w241206,m241201,w341203,...,m263500,n261609,n454800,m264700,n261607,n261503,n261501,n261606,n161605,g431305
68,g321601,0,2,0,0,0,2,0,0,1,...,0,0,0,0,0,2,2,2,0,2
69,g321601,0,2,0,0,0,0,0,0,1,...,0,0,0,0,0,2,2,0,0,2
70,g321601,0,2,0,0,0,0,0,0,1,...,0,0,0,0,0,0,2,0,0,0
71,g321601,0,2,0,0,0,0,0,0,1,...,0,0,0,0,0,0,2,0,0,0
72,g321601,0,2,0,0,0,2,0,0,1,...,0,0,0,0,0,0,2,2,0,2
73,g321602,0,2,0,1,0,0,0,0,0,...,0,0,0,0,0,2,1,2,0,1
74,g321602,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
75,g321602,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
76,g321602,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [88]:
# save starting point for public work with network data
combined_subnets_t.to_csv('data/transposed subnets 2015 python public start.csv', index = False, header = False)

### Public Processing

In [129]:
# setup list of subnets in order that they appear in the dataset
# is: information sharing
# ta: technical assistance provision
# rt: reporting to
# rs: financial resource sharing
# pc: project coordination & collaboration

# in the data set, in the network section, each potential interaction partner is listed 5 times, once for each potential
#  interaction type
# the types of interaction, each forming a separate subnet are listed in the same order for all orgs
# this regularity eases the addition of subnet labels, which is necessary for filtering data by subnets
subnets = ['is',
          'ta',
          'rt',
          'fs',
          'pc']

#### Assign subnet labels and filter data into separate transposed matrices for each subnetwork

In [130]:
# create a list that repeats the subnet list in the correct order and is equal in length to the number of rows in the dataset
subnet_list = ['subnet']
iterations = int( ( combined_subnets_t.shape[0] - 1 ) / 5 )
subnet_iterations = subnets * iterations

for item in subnet_iterations:
    subnet_list.append( item )

# append that subnet list to the dataset for filtering
combined_subnets_t['subnet'] = subnet_list
combined_subnets_t.head(10)

,0,1,2,3,4,5,6,7,8,9,...,85,86,87,88,89,90,91,92,93,subnet
0,anon_id,n281600,g321601,n454701,w231401,w241216,g321612,w241206,m241201,w341203,...,n261609,n454800,m264700,n261607,n261503,n261501,n261606,n161605,g431305,subnet
68,g321601,0,2,0,0,0,2,0,0,1,...,0,0,0,0,2,2,2,0,2,is
69,g321601,0,2,0,0,0,0,0,0,1,...,0,0,0,0,2,2,0,0,2,ta
70,g321601,0,2,0,0,0,0,0,0,1,...,0,0,0,0,0,2,0,0,0,rt
71,g321601,0,2,0,0,0,0,0,0,1,...,0,0,0,0,0,2,0,0,0,fs
72,g321601,0,2,0,0,0,2,0,0,1,...,0,0,0,0,0,2,2,0,2,pc
73,g321602,0,2,0,1,0,0,0,0,0,...,0,0,0,0,2,1,2,0,1,is
74,g321602,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,ta
75,g321602,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,rt
76,g321602,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,fs


In [131]:
# check match at end of the dataframe
combined_subnets_t.tail(10)

,0,1,2,3,4,5,6,7,8,9,...,85,86,87,88,89,90,91,92,93,subnet
1103,m463600,0,1,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,is
1104,m463600,0,1,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,ta
1105,m463600,0,1,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,rt
1106,m463600,0,1,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,fs
1107,m463600,0,1,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,pc
1108,m261502,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,is
1109,m261502,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ta
1110,m261502,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,rt
1111,m261502,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,fs
1112,m261502,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,pc


In [143]:
# filter data and obtain NSQs for each subnetwork

# filter
is_t = combined_subnets_t.loc[combined_subnets_t['subnet'].isin(['subnet','is'])]
ta_t = combined_subnets_t.loc[combined_subnets_t['subnet'].isin(['subnet','ta'])]
pc_t = combined_subnets_t.loc[combined_subnets_t['subnet'].isin(['subnet','pc'])]
rt_t = combined_subnets_t.loc[combined_subnets_t['subnet'].isin(['subnet','rt'])]
fs_t = combined_subnets_t.loc[combined_subnets_t['subnet'].isin(['subnet','fs'])]

# cut final column
is_t = is_t.iloc[:,list(range(is_t.shape[1] - 1))]
ta_t = ta_t.iloc[:,list(range(ta_t.shape[1] - 1))]
pc_t = pc_t.iloc[:,list(range(pc_t.shape[1] - 1))]
rt_t = rt_t.iloc[:,list(range(rt_t.shape[1] - 1))]
fs_t = fs_t.iloc[:,list(range(fs_t.shape[1] - 1))]

# re-transponse
is_nsq = is_t.transpose()
ta_nsq = ta_t.transpose()
pc_nsq = pc_t.transpose()
rt_nsq = rt_t.transpose()
fs_nsq = fs_t.transpose()

# write to file
is_nsq.to_csv('data/InfoShare NSQ 2015 python cleaned.csv', index = False, header = False)
ta_nsq.to_csv('data/TechAssist NSQ 2015 python cleaned.csv', index = False, header = False)
pc_nsq.to_csv('data/PC-C NSQ 2015 python cleaned.csv', index = False, header = False)
rt_nsq.to_csv('data/ReportTo NSQ 2015 python cleaned.csv', index = False, header = False)
fs_nsq.to_csv('data/FinResShare NSQ 2015 python cleaned.csv', index = False, header = False)

#### Transform NSQs into Edgelists

In [163]:
# transform NSQs into Edgelists
is_el = nsq_to_el(is_nsq)
ta_el = nsq_to_el(ta_nsq)
pc_el = nsq_to_el(pc_nsq)
rt_el = nsq_to_el(rt_nsq)
fs_el = nsq_to_el(fs_nsq)

# write data out
is_el.to_csv('data/InfoShare Edgelist 2015 python cleaned.csv', index = False)
ta_el.to_csv('data/TechAssist Edgelist 2015 python cleaned.csv', index = False)
pc_el.to_csv('data/PC-C Edgelist 2015 python cleaned.csv', index = False)
rt_el.to_csv('data/ReportTo Edgelist 2015 python cleaned.csv', index = False)
fs_el.to_csv('data/FinResShare Edgelist 2015 python cleaned.csv', index = False)

#### Reload bimodal networks and transform 'Yes' and 'No' to 1 and 0, respectively

In [180]:
# read data back in to move first row into the column index
# row names must remain in the first column though, due to duplicates

bimodal_sets = pd.read_csv(r'data/bimodal data sets 2015 python public start.csv')
bimodal_sets.head(10)

,anon_id,Winooski_River_watershed,Missisquoi_River_watershed,Tool_Providers__Establish_conservation_easements,Tool_Providers__Enforce_regulation_compliance,Tool_Providers__Communicate_public_information,Tool_Providers__Administer_permits,Tool_Providers__Make_grants,Tool_Providers__Commit_to_cost_sharing,Tool_Providers__Make_loans_and_or_guarantees,...,Domain___Forestry,Domain___Development,Accountability_Mechanism_Feedback_from_federal__state_or_local_elected_officials,Accountability_Mechanism_Feedback_garnered_from_public_hearing__citizen_input_and_other_forms_of_citizen_participation,Accountability_Mechanism_Outcomes_of_court_cases_or_quasi_judicial_rulings,Accountability_Mechanism_Feedback_from_shareholders_or_owners_of_your_organization,Accountability_Mechanism_Feedback_from_consumers_of_your_services,Accountability_Mechanism_Direct_oversight_from_an_administratively_superior_organization_or_unit,Accountability_Mechanism_Professional_codes_of_conduct__principles_of_best_practice,Accountability_Mechanism_Feedback_from_peer_or_partnering_organizations_or_units
0,n281600,Yes,Yes,No,No,Yes,No,Yes,No,No,...,No,No,No,No,No,No,No,No,Yes,No
1,g321601,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,...,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes
2,n454701,Yes,Yes,No,No,No,No,No,No,No,...,No,Yes,Yes,No,No,Yes,Yes,No,Yes,Yes
3,w231401,Yes,No,No,No,Yes,No,Yes,No,No,...,No,Yes,Yes,No,No,No,Yes,No,No,No
4,w241216,Yes,No,Yes,Yes,Yes,Yes,No,Yes,Yes,...,No,Yes,Yes,Yes,No,No,No,No,No,Yes
5,g321612,Yes,Yes,No,Yes,No,Yes,No,No,No,...,No,No,No,Yes,Yes,No,Yes,No,No,Yes
6,w241206,Yes,No,No,Yes,Yes,Yes,No,Yes,No,...,Yes,Yes,Yes,Yes,No,No,No,No,No,Yes
7,m241201,No,Yes,No,Yes,Yes,Yes,No,No,No,...,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,Yes
8,w341203,Yes,No,No,Yes,Yes,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
9,n261608,Yes,Yes,No,No,Yes,No,No,No,No,...,No,No,Yes,No,No,No,Yes,No,Yes,Yes


In [181]:
# basically, do a two-step find replace
# find all cases of "Yes" and change to 1
# find all cases of "No" and change to 0

bimodal_sets.replace(to_replace = {"Yes":1, "No":0}, inplace = True)
bimodal_sets.head(10)

,anon_id,Winooski_River_watershed,Missisquoi_River_watershed,Tool_Providers__Establish_conservation_easements,Tool_Providers__Enforce_regulation_compliance,Tool_Providers__Communicate_public_information,Tool_Providers__Administer_permits,Tool_Providers__Make_grants,Tool_Providers__Commit_to_cost_sharing,Tool_Providers__Make_loans_and_or_guarantees,...,Domain___Forestry,Domain___Development,Accountability_Mechanism_Feedback_from_federal__state_or_local_elected_officials,Accountability_Mechanism_Feedback_garnered_from_public_hearing__citizen_input_and_other_forms_of_citizen_participation,Accountability_Mechanism_Outcomes_of_court_cases_or_quasi_judicial_rulings,Accountability_Mechanism_Feedback_from_shareholders_or_owners_of_your_organization,Accountability_Mechanism_Feedback_from_consumers_of_your_services,Accountability_Mechanism_Direct_oversight_from_an_administratively_superior_organization_or_unit,Accountability_Mechanism_Professional_codes_of_conduct__principles_of_best_practice,Accountability_Mechanism_Feedback_from_peer_or_partnering_organizations_or_units
0,n281600,1,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,g321601,1,1,1,1,1,1,1,1,0,...,0,0,1,1,1,0,1,1,1,1
2,n454701,1,1,0,0,0,0,0,0,0,...,0,1,1,0,0,1,1,0,1,1
3,w231401,1,0,0,0,1,0,1,0,0,...,0,1,1,0,0,0,1,0,0,0
4,w241216,1,0,1,1,1,1,0,1,1,...,0,1,1,1,0,0,0,0,0,1
5,g321612,1,1,0,1,0,1,0,0,0,...,0,0,0,1,1,0,1,0,0,1
6,w241206,1,0,0,1,1,1,0,1,0,...,1,1,1,1,0,0,0,0,0,1
7,m241201,0,1,0,1,1,1,0,0,0,...,1,1,1,1,1,0,1,0,0,1
8,w341203,1,0,0,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
9,n261608,1,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,1,1


In [182]:
# write final results

bimodal_sets.to_csv('data/bimodal data sets 2015 python cleaned.csv', index = False)